# Pipeline to run all files
* ~~All scripts currently output Excel files because we were working independently~~
* ~~<b>We'll be changing this so that each script passes dataframes until the end of the pipeline.</b>~~
* Our plan is to not store anything (refresh daily) but need to consider how to deal with taking in user feedback

In [1]:
#Gather Articles
import NewsAPI as news
import os

#delete any old sorted article files
#if os.path.exists("articles.csv"):
#    os.remove("articles.csv")

#inputs in order: set manual date (yes=1/no=0, no = last 24 hours of news), manual start date as "YYYY-MM-DD"(leave empty if 0), manual end date (leave empty if 0, companies to pull (6= all 19 companies)
articleDB = news.main(1,"2019-02-28","2019-03-03",6) #output is called 'NewsAPIOutput.xlsx' in Python Scripts > Data folder

Gathering articles on (Gap Inc) OR (Foot Locker) OR (L Brands) OR Macerich OR Kimco OR TJX OR CVS OR (Home Depot) OR (Best Buy) OR (Lowe's) OR Walmart OR (Target's) OR TGT OR Amazon OR Kroger OR Walgreens OR Kohl's OR (Dollar General) OR (Bed Bath and Beyond) OR Safeway from: 2019-02-28 to 2019-03-03
536


In [2]:
#Article Cleaning (must pip install tqdm first (only once) to run)
import dataClean as dc
articleDB = dc.DataClean(articleDB)

In [3]:
#Feature Selection and Binary Article Encoding
import FeatureEncoding as fe
contentBinaryMatrix = fe.encoding(0, df=articleDB, text_col='content', norm='wnLemm')
titleBinaryMatrix = fe.encoding(0, df=articleDB, text_col = 'title', norm='wnLemm')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Padmanie\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Padmanie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Padmanie\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Padmanie\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Padmanie\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
content
wnLemm
Binary Encoding
True
title
wnLemm
Binary Encoding
True


In [4]:
#Logistic Regression Classifier + Article Ranking, complete final file is called 'results_encoding.xlsx'
import logReg as lr
articleDB = lr.runLogReg(titleBinaryMatrix, contentBinaryMatrix, articleDB)
#articleDB = runLogReg(titleBinaryMatrix, contentBinaryMatrix, articleDB)

Index([0, 1, 'article_id', 'prediction'], dtype='object')
                                               title  \
0  Tesla's store-shuttering strategy may pull the...   
1  L Brands plans to close roughly 53 Victoria's ...   
2  US stocks rise on hopes for a trade deal with ...   
3  Stocks making the biggest moves midday: Gap, T...   
4    Week ahead: US jobs, retail earnings, ECB rates   

                                         description  
0  Tesla Inc's sudden decision to shutter the bul...  
1  L Brands, owner of Victoria's Secret and Bath ...  
2  NEW YORK (AP) — U.S. stocks are rising Friday,...  
3  These are the stocks posting the largest moves...  
4  While US-China trade talks continue to hold ce...  


In [5]:
# This code extracts and ranks "tags" + counts frequency of tag mentions in articles 
"""
Inputs: 
    1) articleDB - uses column 'content'
    2) (optional) - tag type (could be 'ngrams'{unlimited}, 'bigrams'{terms with up to 2 words}, or 'unigrams'{single terms})
        - default is 'bigrams'
        - future work: add in noun phrases, named entities 
Outputs:
    1) articleDB = articleDB with appended columns "tags" and "tags_top_5"
    2) trendingTermsDB = keyterms by # article mentions
"""
import ContextExtraction as ce
articleDB, trendingTermsDB = ce.retrieveContext(articleDB)

100%|████████████████████████████████████████████████████████████████████████████████| 321/321 [00:52<00:00,  3.75it/s]


In [6]:
import EncodingforRecommender as rec
articleDB = rec.recommender(articleDB)

tifidf Encoding
bin Encoding


100%|███████████████████████████████████████████████████████████████████████████████| 321/321 [00:00<00:00, 635.69it/s]


tf Encoding


100%|███████████████████████████████████████████████████████████████████████████████| 321/321 [00:00<00:00, 587.89it/s]


          san  francisco  tesla    sudden  decision  shutter      bulk  \
0    0.041667   0.071429   1.05  0.166667  0.021739      0.1  0.166667   
1    0.000000   0.000000   0.00  0.000000  0.000000      0.0  0.000000   
2    0.000000   0.000000   0.00  0.000000  0.000000      0.0  0.000000   
3    0.000000   0.000000   0.10  0.000000  0.000000      0.0  0.000000   
4    0.000000   0.000000   0.00  0.000000  0.000000      0.0  0.000000   
5    0.000000   0.000000   0.00  0.000000  0.000000      0.0  0.000000   
6    0.000000   0.000000   0.70  0.000000  0.000000      0.0  0.000000   
7    0.000000   0.000000   0.00  0.000000  0.000000      0.0  0.000000   
8    0.000000   0.000000   0.00  0.000000  0.000000      0.0  0.000000   
9    0.000000   0.000000   0.00  0.000000  0.000000      0.0  0.000000   
10   0.000000   0.000000   0.00  0.000000  0.000000      0.0  0.000000   
11   0.000000   0.000000   0.00  0.000000  0.000000      0.0  0.000000   
12   0.000000   0.000000   0.05  0.000

[321 rows x 16778 columns]


In [7]:
import frontPage as fp
frontpage = fp.FrontPage(articleDB, trendingTermsDB)

In [8]:
import pandas as pd
articles = pd.DataFrame(frontpage['articles'])
topterms= pd.DataFrame(frontpage['topterms'])
articles.columns

Index(['date', 'origContent', 'related_articles', 'source', 'tags', 'title'], dtype='object')

In [9]:
titles = pd.DataFrame(articles['tags'])

In [10]:
pd.DataFrame.to_excel(articleDB, 'Main Version.xlsx')